In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scraped-data3/SentimentAnalysis (1).csv


In [63]:
# installing necessary libraries
!pip install newspaper3k

In [64]:
pip install lxml_html_clean

Note: you may need to restart the kernel to use updated packages.


In [85]:
# this is the code i used  scrape 204 sentences
from newspaper import Article
paths = [
    "https://www.bbc.com/news/live/c1dr7vy39eet",
    "https://www.theguardian.com/environment/2024/dec/17/down-to-earth-long-read-special?utm_source=chatgpt.com",
    "https://www.theguardian.com/lifeandstyle/2025/apr/03/the-leaves-fall-off-but-i-think-thats-normal-the-houseplants-you-just-cant-kill?utm_source=chatgpt.com",
    "https://www.theguardian.com/science/2025/apr/03/eddie-ogorman-obituary?utm_source=chatgpt.com"
]

sentences = []

for path in paths:
    try:
        article = Article(path)
        article.download()
        article.parse()
        
        sentences.extend(article.text.split("."))
    
    except Exception as e:
        print(f"Error processing {path}: {e}")

print(len(sentences))

206


In [86]:
# i tried taking a look at the sentences that were scraped 
for i, sentence in enumerate(sentences):
    print(f"{sentence.strip()}")


Trump touts stock market 'boom' while global markets tumble

Jacqueline Howard

Live reporter

Image source, Reuters Image caption, Trump arriving in Miami earlier today

As investors watched the US stock market take its biggest one-day dive since the pandemic in 2020, the message from the White House to Wall Street was: "Trust in President Trump"
Trump himself said markets would "boom" under his new tariff scheme as the Dow Jones, Nasdaq and S&P plunged over the course of the day
Twenty-four hours after Trump imposed minimum 10% import tariffs worldwide starting on 5 April, the US's key trading partners are mapping out their response
Canadian Prime Minister Mark Carney retaliated with 25% levies on all vehicles imported across the border, lamenting that the close relationship with the US is "now over"
In contrast, Mexican President Claudia Sheinbaum said her country would not retaliate in kind
Meanwhile, the UK has drawn up a 400-page sample list of US goods that could face future tar

In [87]:
# this is to read the scraped data into a csv file called 'scraped_data'
import pandas as pd 
file_path = r"/kaggle/input/scraped-data3/SentimentAnalysis (1).csv"
scraped_data = pd.read_csv(file_path)
scraped_data.head()

,Sentences,Target,AI Labels
0,As investors watched the US stock market take ...,0,0
1,"\nTrump himself said markets would ""boom"" unde...",2,0
2,\n\nTwenty-four hours after Trump imposed mini...,1,1
3,\n\nCanadian Prime Minister Mark Carney retali...,0,0
4,"In contrast, Mexican President Claudia Sheinb...",1,0


In [89]:
# decided to use my manual labelling to train the model,so i had to drop the AI label 
# to avoid interference
scraped_data.drop('AI Labels', axis = 1, inplace = True)

KeyError: "['AI Labels'] not found in axis"

In [90]:
# observing the new scraped_data 
scraped_data.head()

,Sentences,Target
0,As investors watched the US stock market take ...,0
1,"\nTrump himself said markets would ""boom"" unde...",2
2,\n\nTwenty-four hours after Trump imposed mini...,1
3,\n\nCanadian Prime Minister Mark Carney retali...,0
4,"In contrast, Mexican President Claudia Sheinb...",1


In [93]:
# This is the preprocessing stage. Firstly, i want to get rid of all special characters 
# and excess space
import re

scraped_data['Sentences'] = scraped_data['Sentences'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
scraped_data['Sentences'] = scraped_data['Sentences'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

In [94]:
# observing the changes 
scraped_data.head()

,Sentences,Target
0,As investors watched the US stock market take ...,0
1,Trump himself said markets would boom under hi...,2
2,Twentyfour hours after Trump imposed minimum 1...,1
3,Canadian Prime Minister Mark Carney retaliated...,0
4,In contrast Mexican President Claudia Sheinbau...,1


In [95]:
# making all enteries in column 'sentences' a string 
scraped_data['Sentences'] = scraped_data['Sentences'].astype(str)

In [96]:
scraped_data['Sentences'].dtype

dtype('O')

In [74]:
scraped_data.describe

<bound method NDFrame.describe of                                              Sentences  Target
0    As investors watched the US stock market take ...       0
1    Trump himself said markets would boom under hi...       2
2    Twentyfour hours after Trump imposed minimum 1...       1
3    Canadian Prime Minister Mark Carney retaliated...       0
4    In contrast Mexican President Claudia Sheinbau...       1
..                                                 ...     ...
199  When OGorman was made OBE two years later he w...       0
200  In 2018 he received the Pride of Britain lifet...       2
201  By 2021 his health was beginning to deteriorat...       0
202  He is survived by Cliff Sandra and Mark nine g...       0
203  Trump touts stock market boom while global mar...       0

[204 rows x 2 columns]>

In [97]:
# trying to use another model 

import pandas as pd
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

# defining my features 
X = scraped_data['Sentences']
y = scraped_data['Target']

In [76]:
X.head()

,Sentences
0,As investors watched the US stock market take ...
1,Trump himself said markets would boom under hi...
2,Twentyfour hours after Trump imposed minimum 1...
3,Canadian Prime Minister Mark Carney retaliated...
4,In contrast Mexican President Claudia Sheinbau...


In [77]:
y.head()

0    0
1    2
2    1
3    0
4    1
Name: Target, dtype: int64

In [117]:
# splitting my data 
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=15)

#training the model using RandomForest
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('scaler', StandardScaler(with_mean = False)),
    ('random_forest', RandomForestClassifier(n_estimators = 100, random_state = 15))  
])

model = pipeline

model.fit(X_train, y_train)



Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('scaler', StandardScaler(with_mean=False)),
                ('random_forest', RandomForestClassifier(random_state=15))])

In [118]:
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

In [119]:
accuracy = model.score(X_val, y_val)
print(accuracy)

0.5609756097560976


In [122]:
import joblib

with open("Sentiment_Analyzer4.joblib", "wb") as file:
    joblib.dump(pipeline, file)


In [141]:
MOD = joblib.load("Sentiment_Analyzer4.joblib")
MOD.predict(['I was celebrated last week'])


array([1])

In [ ]:
X = scraped_data.drop(columns=['Target']) 
y = scraped_data['Target']

print(X.shape, y.shape)
